In [0]:
-- -- ---------------------------------------------
-- -- 5. Perform SCD-2 MERGE into patient_dim
-- -- ---------------------------------------------
-- MERGE INTO patient_dim AS dim
-- USING patient_final_silver AS src
-- ON dim.patient_id = src.patient_id
-- AND dim.is_current = 'Y'
-- -- ---------------------------------------------
-- -- 5A. Update existing record (close old row)
-- -- ---------------------------------------------
-- WHEN MATCHED AND src.transaction_ind = 'U'
-- THEN UPDATE SET
--     dim.is_current = 'N',
--     dim.effective_end_dt = current_timestamp()
-- -- ---------------------------------------------
-- -- 5B. Insert new version after update
-- -- ---------------------------------------------
-- WHEN NOT MATCHED AND src.transaction_ind = 'U'
-- THEN INSERT (
--       patient_dim_key,
--       patient_id,
--       first_name,
--       last_name,
--       age,
--       address,
--       phone_number,
--       bill_amount,
--       insurance_provider,
--       policy_id,
--       checksum_txt,
--       load_ctl_key,
--       effective_start_dt,
--       effective_end_dt,
--       is_current
-- )
-- VALUES (
--       (SELECT COALESCE(MAX(patient_dim_key), 0) + 1 FROM patient_dim),
--       src.patient_id,
--       src.first_name,
--       src.last_name,
--       src.age,
--       src.address,
--       src.phone_number,
--       src.bill_amount,
--       src.insurance_provider,
--       src.policy_id,
--       src.checksum_txt,
--       src.load_ctl_key,
--       current_timestamp(),
--       '9999-12-31',
--       'Y'
-- )
-- -- ---------------------------------------------
-- -- 5C. Insert new record (Insert case)
-- -- ---------------------------------------------
-- WHEN NOT MATCHED AND src.transaction_ind = 'I'
-- THEN INSERT (
--       patient_dim_key,
--       patient_id,
--       first_name,
--       last_name,
--       age,
--       address,
--       phone_number,
--       bill_amount,
--       insurance_provider,
--       policy_id,
--       checksum_txt,
--       load_ctl_key,
--       effective_start_dt,
--       effective_end_dt,
--       is_current
-- )
-- VALUES (
--       (SELECT COALESCE(MAX(patient_dim_key), 0) + 1 FROM patient_dim),
--       src.patient_id,
--       src.first_name,
--       src.last_name,
--       src.age,
--       src.address,
--       src.phone_number,
--       src.bill_amount,
--       src.insurance_provider,
--       src.policy_id,
--       src.checksum_txt,
--       date_format(current_timestamp(), "yyyyMMddHHmmss"),
--       current_timestamp(),
--       '9999-12-31',
--       'Y'
-- );


In [0]:
-- 5A. Close existing active rows that need update
MERGE INTO patient_dim AS dim
USING patient_final_silver AS src
ON dim.patient_id = src.patient_id
AND dim.is_current = 'Y'
WHEN MATCHED AND src.transaction_ind = 'U'
THEN UPDATE SET
    dim.is_current = 'N',
    dim.effective_end_dt = current_timestamp();


In [0]:
INSERT INTO patient_dim (
      patient_id,
      first_name,
      last_name,
      age,
      address,
      phone_number,
      bill_amount,
      insurance_provider,
      policy_id,
      checksum_txt,
      load_ctl_key,
      effective_start_dt,
      effective_end_dt,
      is_current
)
SELECT
    src.patient_id,
    src.first_name,
    src.last_name,
    src.age,
    src.address,
    src.phone_number,
    src.bill_amount,
    src.insurance_provider,
    src.policy_id,
    src.checksum_txt,
    src.load_ctl_key,
    current_timestamp() AS effective_start_dt,
    TIMESTAMP('9999-12-31 00:00:00') AS effective_end_dt,
    'Y' AS is_current
FROM patient_final_silver src
WHERE src.transaction_ind IN ('I', 'U');


In [0]:
%sql
-- ---------------------------------------------
-- 5. Perform SCD-2 MERGE into insurance_dim
-- ---------------------------------------------
MERGE INTO insurance_dim AS dim
USING insurance_final_silver AS src
ON dim.policy_id = src.policy_id
AND dim.is_current = 'Y'

-- ---------------------------------------------
-- 5A. Update existing record (close old row)
-- ---------------------------------------------
WHEN MATCHED AND src.transaction_ind = 'U'
THEN UPDATE SET
    dim.is_current = 'N',
    dim.effective_end_dt = current_timestamp()

-- ---------------------------------------------
-- 5B. Insert new version after update (Update case)
-- ---------------------------------------------
WHEN NOT MATCHED AND src.transaction_ind = 'U'
THEN INSERT (
      insurance_dim_key,
      policy_id,
      insurance_provider,
      amount_covered,
      claim_status,
      checksum_txt,
      load_ctl_key,
      effective_start_dt,
      effective_end_dt,
      is_current
)
VALUES (
      (SELECT COALESCE(MAX(insurance_dim_key), 0) + 1 FROM insurance_dim),
      src.policy_id,
      src.insurance_provider,
      src.amount_covered,
      src.claim_status,
      src.checksum_txt,
      src.load_ctl_key,
      current_timestamp(),
      '9999-12-31',
      'Y'
)

-- ---------------------------------------------
-- 5C. Insert new record (Insert case)
-- ---------------------------------------------
WHEN NOT MATCHED AND src.transaction_ind = 'I'
THEN INSERT (
      insurance_dim_key,
      policy_id,
      insurance_provider,
      amount_covered,
      claim_status,
      checksum_txt,
      load_ctl_key,
      effective_start_dt,
      effective_end_dt,
      is_current
)
VALUES (
      (SELECT COALESCE(MAX(insurance_dim_key), 0) + 1 FROM insurance_dim),
      src.policy_id,
      src.insurance_provider,
      src.amount_covered,
      src.claim_status,
      src.checksum_txt,
      date_format(current_timestamp(), "yyyyMMddHHmmss"),
      current_timestamp(),
      '9999-12-31',
      'Y'
);


In [0]:
SELECT patient_id, is_current, effective_start_dt, effective_end_dt
FROM patient_dim
ORDER BY patient_id, effective_start_dt;
